In [2]:
!pwd

/home/s2430014/research/enhance-cardsformer-dev


In [3]:
%cd Cardsformer

/home/s2430014/research/enhance-cardsformer-dev/Cardsformer


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [14]:
import numpy as np

In [3]:
# agent_name = "MyAgentSebastianMiller"
agent_name = "LynamicDookaheadAgentV1Master"
# agent_name = "Chimpo"

# 必要コンポーネントの初期化など

In [193]:
import os

# 環境変数を設定
os.environ["PYTHONNET_RUNTIME"] = "coreclr"

# 設定した環境変数を取得
print(os.environ["PYTHONNET_RUNTIME"])

coreclr


In [194]:
import clr

In [195]:
from System import AppDomain
from Env.EnvWrapper import Environment
from Env.Hearthstone import Hearthstone

In [196]:
sabber_asm = None
for asm in AppDomain.CurrentDomain.GetAssemblies():
    if asm.GetName().Name == "SabberStoneAICompetition":
        sabber_asm = asm
        break

if sabber_asm is None:
    raise Exception("SabberStoneAICompetition アセンブリがロードされていません。")

# 3) Type を取得 (完全修飾名)
ld_type = sabber_asm.GetType("SabberStoneBasicAI")

In [197]:
ld_type

In [198]:
sabber_asm

In [199]:
# EnemyAgent.py
import clr
import os
from System import AppDomain, Activator

class EnemyAgent:
    def __init__(self):
        """ C# の DLL をロードして、LynamicDookaheadAgentV1Master をインスタンス化する """
        # 1) SabberStoneCore.dll や SabberStoneAICompetition.dll をロード
        #    パスは環境に合わせて調整してください。
        dll_dir = os.path.join(
            os.getcwd(),
            "../HearthstoneAICompetition/core-extensions/SabberStoneBasicAI/bin/Release/netcoreapp2.1"
        )
        clr.AddReference(os.path.join(dll_dir, "SabberStoneCore.dll"))
        clr.AddReference(os.path.join(dll_dir, "SabberStoneAICompetition.dll"))

        # 2) ロード済みアセンブリを探す
        sabber_asm = None
        for asm in AppDomain.CurrentDomain.GetAssemblies():
            if asm.GetName().Name == "SabberStoneAICompetition":
                sabber_asm = asm
                break

        if sabber_asm is None:
            raise Exception("SabberStoneAICompetition アセンブリがロードされていません。")

        # 3) Type を取得 (完全修飾名)
        ld_type = sabber_asm.GetType("SabberStoneBasicAI.AIAgents.CopyCats." + agent_name)
        if ld_type is None:
            raise Exception(f"Type '{agent_name}' が見つかりません。")

        # 4) Activator でインスタンス化
        self.agent = Activator.CreateInstance(ld_type)

    def get_move(self, po_game):
        """
        C# 側の GetMove(POGame) を呼び出し、PlayerTask を返す
        """
        return self.agent.GetMove(po_game)


In [200]:
clr.AddReference(
    os.getcwd() + "/../dll_files/agent/bin/Release/net8.0/agent.dll"
) #修正

# agent部分の追加

In [201]:
import agent

In [202]:
base = os.getcwd() + "/../HearthstoneAICompetition/core-extensions/SabberStoneBasicAI/bin/Release/netcoreapp2.1"

# 試しに2つの DLL を AddReference
clr.AddReference(os.path.join(base, "SabberStoneCore.dll"))
clr.AddReference(os.path.join(base, "SabberStoneAICompetition.dll"))

# アセンブリ名を確認
# for asm in AppDomain.CurrentDomain.GetAssemblies():
#     print("Loaded assembly:", asm.GetName().Name)

# SabberStoneAICompetition (or 何らかの名前) が見つかったら、そこに PartialObservation があるかチェック
sabber_asm = None
for asm in AppDomain.CurrentDomain.GetAssemblies():
    if asm.GetName().Name == "SabberStoneAICompetition":  # あるいは "SabberStoneBasicAI" かも
        sabber_asm = asm
        break

In [203]:
from System import Activator

po_type = sabber_asm.GetType("SabberStoneBasicAI.PartialObservation.POGame")


In [204]:
# from SabberStoneCore.Visualizer import Visualizer

# prev_game = None
# game = None

# log = []
# game_ui = []
# env_out = []

# hs = Hearthstone(random_cards=True)
# env = Environment(hs)
enemy = EnemyAgent()

# env.initial()
# # state = hs.reset()
# log.append(state)

# for i in range(1000):
#     game = env.Hearthstone.game
#     game_ui.append(Visualizer.Visualize(game))
    
#     act = enemy.get_move(
#         Activator.CreateInstance(po_type, game, False, prev_game)
#     )

#     position, obs, options, done, _, next_state = env.step(act)
#     # state = env.Hearthstone.step(act)

#     env_out.append(out)
#     # log.append(state)
    
#     prev_game = game

#     if done:
#         print("game_end")
#         break
    

# LynamicDookaheadを使った検証  
- 挙動がおかしい部分あり、確認必要

In [208]:
def search_action_index(options, action):
    action_idx = None
    for n, i in enumerate(options):
        if i.ToString() == action.ToString():
            action_idx = n
            break
    if action_idx== None: 
        # print(f"action {action.ToString()} is not in options...\n")

        # print("==== options ====")
        # print([i.ToString() for i in options])

        # print("==== current player options ====")
        # print([i.ToString() for i in game.CurrentPlayer.Options()])
        
        
        return -1

    return action_idx
    

In [10]:
class NpyLogData:
    def __init__(self):
        self.data = {
            'hand_card_names': [],
            'minion_names': [],
            'weapon_names': [],
            'hand_card_scalar': [],
            'minion_scalar': [],
            'hero_scalar': [],
            'next_state_minion_scalar': [],
            'next_state_hero_scalar': [],
        }

    def save_data(self, old_obs, action_idx,next_state):
        self.data['hand_card_names'].append(encoder.encode(old_obs['hand_card_names']).detach().cpu().numpy())
        self.data['minion_names'].append(encoder.encode(old_obs['minion_names']).detach().cpu().numpy())
        self.data['weapon_names'].append(encoder.encode(old_obs['weapon_names']).detach().cpu().numpy())
        self.data['hand_card_scalar'].append(old_obs['hand_card_scalar_batch'][action_idx].detach().cpu().numpy().astype(np.int64))
        self.data['minion_scalar'].append(old_obs['minion_scalar_batch'][action_idx].detach().cpu().numpy().astype(np.int64))
        self.data['hero_scalar'].append(old_obs['hero_scalar_batch'][action_idx].detach().cpu().numpy().astype(np.int64))
        self.data['next_state_minion_scalar'].append(next_state['minion_scalar'].detach().cpu().numpy().astype(np.int64))
        self.data['next_state_hero_scalar'].append(next_state['hero_scalar'].detach().cpu().numpy().astype(np.int64))

    def save_to_npy(self, npy_name):
        np.save('off_line_data_vs_ai' + str(j) + '.npy', self.data)
    

In [230]:
import random
import numpy as np
from Algo.utils import log

from Algo.encoder import Encoder
from transformers import AutoTokenizer, AutoModel



tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
auto_model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")
encoder = Encoder(model=auto_model, tokenizer=tokenizer)
encoder.to('cpu')

log.info('Data generation begins.')
npy_log_data = NpyLogData()

for j in range(1):
    game = Hearthstone(random_cards=True)
    env = Environment(game)
    
    # data = {
    #     'hand_card_names': [],
    #     'minion_names': [],
    #     'weapon_names': [],
    #     'hand_card_scalar': [],
    #     'minion_scalar': [],
    #     'hero_scalar': [],
    #     'next_state_minion_scalar': [],
    #     'next_state_hero_scalar': [],
    # }
    position, obs, options, done, episode_return = env.initial()
    for i in range(10):
        if i % 100 == 0:
            print(f"match {i} finished")
        game = None
        prev_game = None
        enemy = EnemyAgent()

        invalid_cnt = 0
        while True:
            num_options = len(options)
            # action_idx = random.randint(0, num_options - 1)
            # action = options[action_idx]

            game = env.Hearthstone.game

            action = enemy.get_move(
                Activator.CreateInstance(po_type, game, False, game)
            )
            # prev_game = env.Hearthstone.game

            
            action_idx = search_action_index(options, action)
            if action_idx == -1:
                num_options = len(options)
                action_idx = random.randint(0, num_options - 1)
                invalid_cnt +=1
            
            player = position
            old_obs = obs
            
            position, obs, options, done, _, next_state = env.step(action, player)

            npy_log_data.save_data(old_obs, action_idx, next_state)
            # data['hand_card_names'].append(encoder.encode(old_obs['hand_card_names']).detach().cpu().numpy())
            # data['minion_names'].append(encoder.encode(old_obs['minion_names']).detach().cpu().numpy())
            # data['weapon_names'].append(encoder.encode(old_obs['weapon_names']).detach().cpu().numpy())
            # data['hand_card_scalar'].append(old_obs['hand_card_scalar_batch'][action_idx].detach().cpu().numpy().astype(np.int64))
            # data['minion_scalar'].append(old_obs['minion_scalar_batch'][action_idx].detach().cpu().numpy().astype(np.int64))
            # data['hero_scalar'].append(old_obs['hero_scalar_batch'][action_idx].detach().cpu().numpy().astype(np.int64))
            # data['next_state_minion_scalar'].append(next_state['minion_scalar'].detach().cpu().numpy().astype(np.int64))
            # data['next_state_hero_scalar'].append(next_state['hero_scalar'].detach().cpu().numpy().astype(np.int64))
            if done:
                break

            
    npy_log_data.save_to_npy('off_line_data_vs_ai' + str(j) + '.npy')


[INFO:2479335 1693783427:15 2025-01-02 15:05:09,424] Data generation begins.
[INFO:2479335 1693783427:15 2025-01-02 15:05:09,424] Data generation begins.
[INFO:2479335 1693783427:15 2025-01-02 15:05:09,424] Data generation begins.
[INFO:2479335 1693783427:15 2025-01-02 15:05:09,424] Data generation begins.
[INFO:2479335 1693783427:15 2025-01-02 15:05:09,424] Data generation begins.
[INFO:2479335 1693783427:15 2025-01-02 15:05:09,424] Data generation begins.
[INFO:2479335 1693783427:15 2025-01-02 15:05:09,424] Data generation begins.
[INFO:2479335 1693783427:15 2025-01-02 15:05:09,424] Data generation begins.


match 0 finished


# visualizerの追加

In [38]:
print(game_ui[0])

+-Player1   ---------------------------------------------------------[o.........]-+---------+
¦                                     o-----+                                     ¦         ¦
¦                                     | 0/30|                                     ¦ +-----+ ¦
¦                                     |     |                                     ¦ |     | ¦
¦                                     |  /Po| SteaSho                             ¦ |  0  | ¦
¦                                     +-----+                                     ¦ | SECR| ¦
¦                                     Rexxar   <<==                               ¦ +-----+ ¦
+-HAND-v--------------------------------------------------------------------------+---------+
¦                         HuntMar StraTig Houndma AnimCom                         ¦         ¦
¦                         +--0-S+ o--5-M+ o--4-M+ +--3-S+                         ¦ +-----+ ¦
¦                         |Chang| | 5/5 | | 4/3 | |Summo|   

In [19]:
log

[('Player1',
  {'hand_card_names': ['Guardian of Kings',
    'Humility',
    'Earthen Ring Farseer',
    'Aldor Peacekeeper',
    'Reinforce',
    None,
    None,
    None,
    None,
    None,
    None],
   'minion_names': [None,
    None,
    None,
    None,
    None,
    None,
    None,
    None,
    None,
    None,
    None,
    None,
    None,
    None],
   'weapon_names': [None, None],
   'secret_names': [None, None, None, None, None],
   'hand_card_scalar_batch': array([[[7., 7., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 1., 0., 0., 0.],
           [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 1., 0., 0.],
           [3., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 1., 0., 0., 0.],
           [3., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 1., 0., 0., 0.],
           [2., 2., 0., 0., 0., 0., 0., 0., 0

# policy modelでの検証

In [11]:
import os

# 環境変数を設定
os.environ["PYTHONNET_RUNTIME"] = "coreclr"

# 設定した環境変数を取得
print(os.environ["PYTHONNET_RUNTIME"])

from typing import OrderedDict
from Env.Hearthstone import Hearthstone
from Env.EnvWrapper import Environment
from Model.PredictionModel import PredictionModel
from Model.ModelWrapper import Model as PolicyModel

from transformers import AutoModel, AutoTokenizer
import torch
from Algo.encoder import Encoder
import pandas as pd

import logging

# shandle = logging.StreamHandler()
# shandle.setFormatter(
#     logging.Formatter(
#         '[%(levelname)s:%(process)d %(module)s:%(lineno)d %(asctime)s] '
#         '%(message)s'))
# log = logging.getLogger('Cardsformer')
# log.propagate = False
# log.addHandler(shandle)
# log.setLevel(logging.INFO)


name_list = pd.read_csv('Env/classical_cards.csv')['name'].tolist()

# NUM_ROUNDS = 1
# checkpoint_path = "./trained_policy_model/Cardsformer/Trained_weights_1000000.ckpt"
device_number = 'cpu'


# model =PolicyModel(device=device_number)
# checkpoint_states = torch.load(checkpoint_path,map_location=torch.device('cpu'))
# model.get_model().load_state_dict(checkpoint_states)

prediction_model = PredictionModel()
# prediction
checkpoint_states = torch.load("./trained_models/prediction_model4715.tar", map_location="cpu")['model_state_dict']

# unwrap the prediction model
new_state_dict = OrderedDict()
for k, v in checkpoint_states.items():
    name = k[7:]
    new_state_dict[name] = v

prediction_model.load_state_dict(new_state_dict)
prediction_model.to("cpu")
prediction_model.eval()

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
auto_model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")
encoder = Encoder(model=auto_model, tokenizer=tokenizer)
encoder.to(device)

coreclr


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [12]:
position, obs, options, done, episode_return = env.initial()

In [ ]:
win = [0, 0]

npy_log_data = NpyLogData()

for i in range(1000):
    if i % 100 == 0:
        print(f"match {i} finished")
    
    while True:
        
        num_options = len(options)
    
        # hand_card_embed = encoder.encode(obs['hand_card_names'])
        # minion_embed = encoder.encode(obs['minion_names'])
        # weapon_embed = encoder.encode(obs['weapon_names'])
        # secret_embed = encoder.encode(obs['secret_names'])
        with torch.no_grad():
    
            hand_card_embed = encoder.encode(obs['hand_card_names'])
            minion_embed = encoder.encode(obs['minion_names'])
            weapon_embed = encoder.encode(obs['weapon_names'])
            secret_embed = encoder.encode(obs['secret_names'])
            with torch.no_grad():
                next_state = prediction_model([hand_card_embed, minion_embed, weapon_embed, obs['hand_card_scalar_batch'], obs['minion_scalar_batch'], obs['hero_scalar_batch']])
            obs['next_minion_scalar'] = next_state[0]
            obs['next_hero_scalar'] = next_state[1]
            with torch.no_grad():
                agent_output = model.forward(hand_card_embed, minion_embed, secret_embed, weapon_embed, obs, num_options, actor = True)
            # uncomment the following line to see the action-value results of each available action
            # for i in range(num_options):
            #     log.info('--ACTION-- {} --VALUE-- {}'.format(options[i].FullPrint(), agent_output.reshape(-1)[i]))
            agent_output = agent_output.argmax()
            action_idx = int(agent_output.cpu().detach().numpy())
        action = options[action_idx]
    
        player = position
        old_obs = obs
        # log.info(action.FullPrint())  # print the performing action
        position, obs, options, done, episode_return, next_state = env.step(action, player)
        # log.info(env.Hearthstone.game.FullPrint())  # print current game state
    
        
        npy_log_data.save_data(old_obs, action_idx, next_state)
        
        if done:
            if episode_return > 0:
                win[0] += 1
            elif episode_return < 0:
                win[1] += 1
            else:
                log.info("No winner???")
            break


    
npy_log_data.save_to_npy('off_line_data_vs_policy_model.npy')

match 0 finished
match 100 finished


In [ ]:
position, obs, options, done, _, next_state = env.step(action, player)